In [1]:
from resize_image import *
from Image import load_img, save_img, Image, clamp_image
from filter_image import *

from locale import ABDAY_1
from matplotlib.colors import hex2color
from Image import Image

In [7]:
class make_box_filter(object):
    def __init__(self, kernel_size):
        if kernel_size % 2 == 0:
            raise ValueError("kernel size needs to be odd")
        self.kernel = Image(kernel_size, kernel_size, 1)
        self.h = self.kernel.h
        self.w = self.kernel.w
        self.c = self.kernel.c
        self.ret = Image(self.h, self.w, self.c)
        for j in range(self.ret.w):
            for i in range(self.ret.h):
                ## Fill all with 1 and normalize
                self.ret[i,j,0]=1/(self.ret.w*self.ret.h)

    def __call__(self, local):
        out = 0


        return out

def padding_image(img, filter):
    stride = 1
    padding = int((filter.h - stride)/2)
    ret = Image(img.h+padding*2, img.w+padding*2, img.c)
    
    for k in range(ret.c):
        for j in range(padding,img.w+padding):
            for i in range(padding,img.h+padding):
                ret[i,j,k] = img[i-padding,j-padding,k]

    return ret

In [56]:
def convolve_image(img, filter, preserve):
    if filter.c != 1:
        raise ValueError("filter kernel should only have 1 channel")

    if preserve == True:
        ret = Image(img.h-filter.h +1, img.w-filter.w +1, img.c)
        for k in range(ret.c):
            for j in range(ret.w):
                for i in range(ret.h):
                    value = 0
                    for n in range(filter.w):
                        for m in range(filter.h):
                            temp = filter.ret[m,n,0]*img[m+i,n+j,k]
                            value = value+temp
                    ret[i,j,k] = value

    if preserve == False:
        ret = Image(img.h-filter.h +1, img.w-filter.w +1, 1)
        for j in range(ret.w):
            for i in range(ret.h):
                value = 0
                for k in range(img.c):
                    for n in range(filter.w):
                        for m in range(filter.h):
                            temp = filter.ret[m,n,0]*img[m+i,n+j,k]
                            value = value+temp
                ret[i,j,0] = value

    return ret

In [66]:
class make_highpass_filter(object):
    def __init__(self):
        self.kernel = Image(3, 3, 1)

        self.h = self.kernel.h
        self.w = self.kernel.w
        self.c = self.kernel.c

        self.ret = Image(self.h, self.w, self.c)
        self.ret[0,0,0] = self.ret[0,2,0] = self.ret[2,0,0] = self.ret[2,2,0] = 0
        self.ret[1,0,0] = self.ret[0,1,0] = self.ret[2,1,0] = self.ret[1,2,0] = -1
        self.ret[1,1,0] = 4
                
    def __call__(self, local):
        out = 0


        return out

class make_sharpen_filter(object):
    def __init__(self):
        self.kernel = Image(3, 3, 1)

        self.h = self.kernel.h
        self.w = self.kernel.w
        self.c = self.kernel.c

        self.ret = Image(self.h, self.w, self.c)
        self.ret[0,0,0] = self.ret[0,2,0] = self.ret[2,0,0] = self.ret[2,2,0] = 0
        self.ret[1,0,0] = self.ret[0,1,0] = self.ret[2,1,0] = self.ret[1,2,0] = -1
        self.ret[1,1,0] = 5

    def __call__(self, local):
        out = 0


        return out

class make_emboss_filter(object):
    def __init__(self):
        self.kernel = Image(3, 3, 1)

        self.h = self.kernel.h
        self.w = self.kernel.w
        self.c = self.kernel.c

        self.ret = Image(self.h, self.w, self.c)
        self.ret[0,0,0] = -2
        self.ret[1,0,0] = self.ret[0,1,0] = -1
        self.ret[2,0,0] = self.ret[0,2,0] = 0
        self.ret[1,1,0] = self.ret[2,1,0] = self.ret[1,2,0] = 1
        self.ret[2,2,0] = 2

    def __call__(self, local):
        out = 0


        return out


In [52]:
img = load_img("data/dog.jpg")
filter = make_box_filter(7)
pad_img = padding_image(img, filter)
blur = convolve_image(pad_img, filter, True)
clamp_image(blur)
save_img("out/convolution_image.jpg", blur)

In [57]:
img = load_img("data/dog.jpg")
filter = make_box_filter(7)
pad_img = padding_image(img, filter)
blur = convolve_image(pad_img, filter, False)
clamp_image(blur)
save_img("out/convolution_no_preserve_image.jpg", blur)

In [63]:
    img = load_img("data/dog.jpg")
    filter = make_highpass_filter()
    pad_img = padding_image(img, filter)
    blur = convolve_image(pad_img, filter, False)
    clamp_image(blur)
    save_img("out/highpass_case.jpg", blur)

In [65]:
    img = load_img("data/dog.jpg")
    filter = make_sharpen_filter()
    pad_img = padding_image(img, filter)
    blur = convolve_image(pad_img, filter, True)
    clamp_image(blur)
    save_img("out/sharpen_case.jpg", blur)

In [ ]:
    img = load_img("data/dog.jpg")
    filter = make_emboss_filter()
    pad_img = padding_image(img, filter)
    blur = convolve_image(pad_img, filter, True)
    clamp_image(blur)
    save_img("out/emboss_case.jpg", blur)

In [63]:
def nearest_resize(img, w, h):
    source = img
    target = Image(h, w, img.c)
    ## note: access pixel should input [h_coord, w_coord, c_coord], and simply use int() to get integer number, not round()
    ## TODO: use source pixel value to fill target pixel value
    
    ## Get ratio for resize
    w_ratio = w/source.w ; h_ratio = h/source.h
    w_ratio_array = []; h_ratio_array = []; 

    ## Caulculate the integer index of each target pixel
    for j in range(w):
        w_ratio_array.append(int((j+0.5)/w_ratio))
    for i in range(h):
        h_ratio_array.append(int((i+0.5)/h_ratio))

    ## Read source pixel by index
    for k in range(target.c):
        for j in range(w):
            for i in range(h):
                target[i,j,k] = source[h_ratio_array[i],w_ratio_array[j],k] 

    return target

In [42]:
def bilinear_resize(img, w, h):
    source = img
    target = Image(h, w, img.c)
    ## note: access pixel should input [h_coord, w_coord, c_coord], and simply use int() to get integer number, not round()
    ## TODO: use source pixel value to fill target pixel value
    
    import numpy as np
    ## Get ratio for resize
    w_ratio = (w-1)/(source.w-1) ; h_ratio = (h-1)/(source.h-1)
    w_ratio_array = []; h_ratio_array = []; 

    for j in range(w):
        w_ratio_array.append(round(j/w_ratio,10))
    for i in range(h):
        h_ratio_array.append(round(i/h_ratio,10))
    
    for k in range(target.c):
        for j in range(w):
            for i in range(h):
                height1 = (h_ratio_array[i]-np.floor(h_ratio_array[i]))
                height2 = 1-height1
                width1  = (w_ratio_array[j]-np.floor(w_ratio_array[j]))
                width2  = 1-width1
                ratio1 = (1-height1)*(1-width1)
                ratio2 = (1-height1)*(1-width2)
                ratio3 = (1-height2)*(1-width1)
                ratio4 = (1-height2)*(1-width2)
                value1 = ratio1 * source[int(np.floor(h_ratio_array[i])),int(np.floor(w_ratio_array[j])),k] 
                value2 = ratio2 * source[int(np.floor(h_ratio_array[i])),int(np.ceil(w_ratio_array[j])),k] 
                value3 = ratio3 * source[int(np.ceil(h_ratio_array[i])) ,int(np.floor(w_ratio_array[j])),k] 
                value4 = ratio4 * source[int(np.ceil(h_ratio_array[i])) ,int(np.ceil(w_ratio_array[j])),k] 
                target[i,j,k] = value1+value2+value3+value4
                
    return target


In [55]:
source[0,0,0]

0.2313

In [64]:
img = load_img("data/dogsmall.jpg")
resized = nearest_resize(img, img.w * 4, img.h * 4)
save_img("out/nn_resize_4x.jpg", resized)

img = load_img("data/dog.jpg")
resized = nearest_resize(img, 713, 467)
save_img("out/nn_resize_adaptive.jpg", resized)

In [43]:
img = load_img("data/dogsmall.jpg")
resized = bilinear_resize(img, img.w * 4, img.h * 4)
save_img("out/bl_resize_4x.jpg", resized)

In [44]:
img = load_img("data/dog.jpg")
resized = bilinear_resize(img, 713, 467)
save_img("out/bl_resize_adaptive.jpg", resized)

In [75]:
class make_box_filter(object):
    def __init__(self, kernel_size):
        if kernel_size % 2 == 0:
            raise ValueError("kernel size needs to be odd")
        self.kernel = Image(kernel_size, kernel_size, 1)
        self.h = self.kernel.h
        self.w = self.kernel.w
        self.c = self.kernel.c


    def __call__(self, local):

        for j in range(self.w):
            for i in range(self.h):
                ## Fill all with 1 and normalize
                kernel[i,j,0]=1/(kernel.w*kernel.h)
        return kernel

In [77]:
    img = load_img("data/dog.jpg")
    filter = make_box_filter(47)
    pad_img = padding_image(img, filter)

In [74]:
filter[0]

TypeError: 'make_box_filter' object is not subscriptable